In [2]:
#cleaning data process.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import urllib
import requests
import pymongo
from bs4 import BeautifulSoup as bs1
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import FreqDist
import seaborn as sbn
from textblob import TextBlob, Word, Blobber
from nltk.stem import PorterStemmer
st = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noahm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean(txt):
    txt = txt.str.replace("(<br/>)", "")
    txt = txt.str.replace('(<a).*(>).*(</a>)', '')
    txt = txt.str.replace('(&amp)', '')
    txt = txt.str.replace('(&gt)', '')
    txt = txt.str.replace('(&lt)', '')
    txt = txt.str.replace('(\xa0)', ' ')  
    return txt

In [4]:
mydata=pd.read_csv('train.csv')

In [5]:
mydata.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
mydata['text'].astype(str)

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [7]:
mydata=mydata[~mydata['text'].isnull()]

In [8]:
mydata['text']=clean(mydata['text'])

mydata['cleaned'] = mydata['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
mydata['cleaned'] = mydata['cleaned'].str.replace('[^\w\s]', '')


stop = stopwords.words('english')
mydata['cleaned'] = mydata['cleaned'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


<ipython-input-3-2deb8f962941>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  txt = txt.str.replace("(<br/>)", "")
<ipython-input-3-2deb8f962941>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  txt = txt.str.replace('(<a).*(>).*(</a>)', '')
<ipython-input-3-2deb8f962941>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  txt = txt.str.replace('(&amp)', '')
<ipython-input-3-2deb8f962941>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  txt = txt.str.replace('(&gt)', '')
<ipython-input-3-2deb8f962941>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  txt = txt.str.replace('(&lt)', '')
<ipython-input-3-2deb8f962941>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  txt = txt.str.replace('(\xa0)'

In [9]:
from nltk.stem import PorterStemmer
st = PorterStemmer()
mydata['cleaned'] = mydata['cleaned'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [10]:
mydata.head()

,id,keyword,location,text,target,cleaned
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may allah forgiv us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resid ask shelter place notifi offic evacu she...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 peopl receiv wildfir evacu order california
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...


In [17]:
from sklearn.model_selection import train_test_split
import config
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

In [12]:
X_train, X_test, y_train, y_test = train_test_split(mydata.cleaned, mydata.target, test_size=0.2, random_state=0)
count_vec = CountVectorizer()

In [13]:
train_vecs = count_vec.fit_transform(X_train)
test_vecs = count_vec.transform(X_test)
train=xgb.DMatrix(train_vecs, label=y_train)
test=xgb.DMatrix(test_vecs, label=y_test)

In [14]:
param={
    'max_depth':6,
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class' : 2,
    'verbosity':0
}
epochs=1000

In [15]:
model = xgb.train(param, train, epochs)

In [18]:
predictions=model.predict(test)
n1=accuracy_score(y_test, predictions)

In [19]:
model2=xgb.XGBClassifier(use_label_encoder=False, verbosity = 0)
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
results = cross_val_score(model2, train_vecs, y_train, cv=kfold)
model2.fit(train_vecs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

In [20]:
predictions2=model2.predict(test_vecs)

In [21]:
#model 2 with stratified cross validation
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, random_state=7, shuffle=True)
results2 = cross_val_score(model2, train_vecs, y_train, cv=kfold)

In [22]:
print("Accuracy: %.2f%%" % (n1*100))
print("Accuracy: %.2f%%" % (results2.mean()*100))
print("Accuracy: %.2f%%" % (results.mean()*100))

Accuracy: 79.51%
Accuracy: 78.59%
Accuracy: 78.05%


In [30]:
comparisondf=pd.DataFrame()
comparisondf['tweet']=X_test
comparisondf['actual']=y_test
comparisondf['predicted']=predictions2

In [32]:
comparisondf.head()

,tweet,actual,predicted
311,katiekatcub alreadi know shit goe world seri a...,0,0
4970,lemairele danharmon peopl near meltdown comic ...,0,1
527,16 tix calgari flame vs col avalanch preseason...,0,1
6362,ever think run choic life rembr there kid choi...,0,1
800,dotish blight car go right ahead mine,0,0
